In [1]:
import sys 
sys.path.append('../../')

In [3]:
from main_multitask import LLM_Multitask

In [5]:
model = LLM_Multitask.load_from_checkpoint('../../models/product_title_multitask/version_1/epoch=0-step=349636.ckpt', 
    label_map_file='../../datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt')

INFO:root:Unused kwargs when getting t5-base: {'distance_func': 'cosine', 'loss_type': 'cross-entropy', 'margin': None, 'hidden_states_type': 'encoder-last', 'add_simcse': True, 'label_map_file': '../../datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt', 'label_weight_type': 'ancestor-high', 'additional_tricks': None, 'multitask_names': ['clm_ali_title2tax', 'dlm_wish_title', 'emb_all_title2tax', 'seqclf_ali_title2tax'], 'auto_task_weight': False}
/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instan

In [6]:
model.eval()
tmp = 2

In [22]:
# denoise: Autumn and Winter Women's Fashion O-Neck Long Sleeve Velvet Tops Casual Solid Color Pullover Sweatshirts Loose Plus Size Blouses
model.tokenizer.batch_decode(
    model.transformer.generate(
        **model.tokenizer(("Denoise product: Autumn and Winter <extra_id_0>'s Fashion O-Neck Long Sleeve Velvet Tops Casual"
        " Solid Color <extra_id_1> Loose Plus Size <extra_id_2>"), 
            return_tensors='pt'), length_penalty=0, max_new_tokens=50, num_beams=3 ,num_return_sequences=1,
            do_sample=False
    ), skip_special_tokens=False
)

["<pad><extra_id_0> Women's<extra_id_1> Shirts<extra_id_2> S-5XL<extra_id_3></s>"]

In [23]:
# clm-based categorize: Autumn and Winter Women's Fashion O-Neck Long Sleeve Velvet Tops Casual Solid Color Pullover Sweatshirts Loose Plus Size Blouses
model.tokenizer.batch_decode(
    model.transformer.generate(
        **model.tokenizer(("Top-down categorize product: Autumn and Winter Women's Fashion O-Neck Long Sleeve Velvet Tops Casual"
        " Solid Color Pullover Sweatshirts Loose Plus Size Blouses"), 
            return_tensors='pt'), length_penalty=0, max_new_tokens=50, num_beams=3 ,num_return_sequences=1,
            do_sample=False
    ), skip_special_tokens=False
)

["<pad> women's clothing > active & loungewear > hoodies & sweatshirts</s>"]

In [42]:
# seqclf-based categorize product title
label_map_rev = {model.label_map[i]: i for i in model.label_map}

inputs = model.tokenizer("Classify product: Autumn and Winter Women's Fashion O-Neck Long Sleeve Velvet Tops Casual Solid Color Pullover Sweatshirts Loose Plus Size Blouses", 
    return_tensors='pt')
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
hidden_states = model.get_hidden_states(
    input_ids = input_ids, 
    attention_mask = attention_mask
)
logits = model.clf_head(hidden_states)
prediction = logits
probs = prediction.sigmoid()
top_probs, top_pred_indices = probs.topk(probs.size(1))
assert prediction.size(1) == len(model.label_map)

In [43]:
[(label_map_rev[i], probs[0][i].item()) for i in top_pred_indices.detach().cpu().numpy().reshape(-1)[:10]]

[("women's clothing", 0.6990596652030945),
 ("women's clothing > activewear & loungewear > hoodies & sweatshirts",
  0.31234562397003174),
 ("women's clothing > tops", 0.1961180716753006),
 ("women's clothing > tops > blouses & shirts", 0.16454944014549255),
 ("women's clothing > sweaters > sweaters", 0.1362212598323822),
 ("women's clothing > sweaters", 0.09625529497861862),
 ("women's clothing > activewear & loungewear", 0.09437346458435059),
 ('sports > sports clothing', 0.0761471688747406),
 ('mother & kids', 0.07304281741380692),
 ('sports > sports clothing > shirts', 0.06922844052314758)]

In [48]:
# seqclf-based 0-shot categorize ood taxonomy from google shopping
label_map_rev = {model.label_map[i]: i for i in model.label_map}

inputs = model.tokenizer("Classify taxonomy: Animals & Pet Supplies > Pet Supplies > Bird Supplies > Bird Cage Accessories", 
    return_tensors='pt')
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
hidden_states = model.get_hidden_states(
    input_ids = input_ids, 
    attention_mask = attention_mask
)
logits = model.clf_head(hidden_states)
prediction = logits
probs = prediction.sigmoid()
top_probs, top_pred_indices = probs.topk(probs.size(1))
assert prediction.size(1) == len(model.label_map)

In [49]:
[(label_map_rev[i], probs[0][i].item()) for i in top_pred_indices.detach().cpu().numpy().reshape(-1)[:10]]

[('home & garden > pet products', 0.991324782371521),
 ('home & garden', 0.9824209809303284),
 ('home & garden > pet products > bird supplies', 0.9470421671867371),
 ('home & garden > pet products > bird supplies > bird cages & nests',
  0.24729923903942108),
 ('home & garden > pet products > bird supplies > birdcage covers',
  0.0858282744884491),
 ('home & garden > pet products > bird supplies > bird training',
  0.05079670995473862),
 ('home & garden > pet products > bird supplies > bird perches',
  0.040641505271196365),
 ('home & garden > pet products > bird supplies > bird toys',
  0.0399768091738224),
 ('home & garden > pet products > bird supplies > birdcage stands',
  0.026033060625195503),
 ('home & garden > garden supplies', 0.021642860025167465)]

In [71]:
# emb visual
import dvc.api 
import pandas as pd
from torch import nn
df_test = pd.read_json(dvc.api.get_url(
    'data/wish_products_internallabel/wish_products_offshore_labelled_validated_processed.json',
    repo='git@github.com:junwang-wish/query_understanding_data.git'
), lines=True)
inputs = model.tokenizer(('Embed product: ' + df_test.title).tolist(), 
    return_tensors='pt', padding="max_length", truncation=True, max_length=50)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
hidden_states = model.get_hidden_states(
    input_ids = input_ids, 
    attention_mask = attention_mask
)
hidden_states_norm = nn.functional.normalize(hidden_states)

In [74]:
import umap
import umap.plot
import numpy as np
umap.plot.output_notebook()
hover_data = pd.DataFrame({'index': np.arange(len(df_test)),
                           'title': df_test['title'],
                           'text': df_test['category'].apply(tuple)
                           })
mapper = umap.UMAP().fit(hidden_states.detach().cpu().numpy())
fig = umap.plot.interactive(
    mapper, 
    labels=df_test['category'].apply(lambda x: x[0]), 
    point_size=5,
    hover_data=hover_data
)

Loading BokehJS ...

In [75]:
umap.plot.show(fig)